In [1]:
!pip install transformers==4.41.2 peft==0.11.1 accelerate==0.30.1 bitsandbytes==0.43.1 --force-reinstall

^C


In [3]:
import torch
import requests
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel


BASE_MODEL = "meta-llama/Meta-Llama-3.1-8B"

HF_USER = "ed-donner"
PROJECT_NAME = "pricer"
RUN_NAME = "2024-09-13_13.04.39"
FINETUNED_MODEL = f"{HF_USER}/{PROJECT_NAME}-{RUN_NAME}"
REVISION = "e8d637df551603dc86cd7a1598a8f44af4d7ae36"


In [1]:
import json
import requests
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import torch

HF_TOKEN = "hf_xxxxxxxxxxxxxxxxx"
BASE_MODEL = "meta-llama/Meta-Llama-3.1-8B"

HF_USER = "ed-donner"
PROJECT_NAME = "pricer"
RUN_NAME = "2024-09-13_13.04.39"
FINETUNED_MODEL = f"{HF_USER}/{PROJECT_NAME}-{RUN_NAME}"
REVISION = "e8d637df551603dc86cd7a1598a8f44af4d7ae36"

# -------------------------------------------
# 1) DOWNLOAD RAW config.json WITHOUT parsing
# -------------------------------------------
url = f"https://huggingface.co/{BASE_MODEL}/resolve/main/config.json"
headers = {"Authorization": f"Bearer {HF_TOKEN}"}
raw = requests.get(url, headers=headers).json()

print("Original rope_scaling from HF:")
print(raw.get("rope_scaling"))


W1126 01:57:50.168000 28832 torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
C:\Users\sushm\AppData\Roaming\Python\Python311\site-packages\transformers\models\auto\configuration_auto.py:919: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
C:\Users\sushm\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


ValueError: `rope_scaling` must be a dictionary with two fields, `type` and `factor`, got {'factor': 8.0, 'low_freq_factor': 1.0, 'high_freq_factor': 4.0, 'original_max_position_embeddings': 8192, 'rope_type': 'llama3'}

In [ ]:
QUESTION = "How much does this cost to the nearest dollar?"
PREFIX = "Price is $"

def predict_price_usd(description: str):
    prompt = f"{QUESTION}\n{description}\n{PREFIX}"

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=15,
        temperature=0.2
    )

    decoded = tokenizer.decode(outputs[0])

    # Extract number
    text = decoded.split("Price is $")[-1].replace(",", "")
    match = re.search(r"[-+]?\d*\.?\d+", text)
    return float(match.group()) if match else None


def usd_to_inr(amount_usd: float):
    data = requests.get("https://api.exchangerate-api.com/v4/latest/USD").json()
    rate = data["rates"]["INR"]
    return amount_usd * rate, rate


In [ ]:
desc = input("Enter product description:\n")

usd = predict_price_usd(desc)

if usd is None:
    print("Model could not extract price.")
else:
    inr, rate = usd_to_inr(usd)
    print("\n--- Price Prediction ---")
    print(f"USD Price: ${usd:.2f}")
    print(f"USD → INR Rate: {rate:.2f}")
    print(f"Price in INR: ₹{inr:.2f}")


In [3]:
import chromadb

client = chromadb.PersistentClient(path="products_vectorstore")

collections = client.list_collections()

print("Available collections:")
for c in collections:
    print(" -", c.name)


Available collections:
 - products


In [4]:
import chromadb

client = chromadb.PersistentClient(path="products_vectorstore")  # your folder

# list collections
print(client.list_collections())

# open collection
collection = client.get_collection("products")  # change to your collection name

# get full data
data = collection.get(include=["documents", "metadatas", "embeddings"])

print(data)


[Collection(id=b5403c4d-8e45-45c9-a225-d5db4d225791, name=products)]


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

